# Neural Network for inpainting one pixel an image using tensorflow 1

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib as plt
from data_generators import gen_frm_pic
from util_functions import util_functions

INDEX_TRGT = 12
INDEX_END = 20
ROW_SHAPE = (1,25)

## Generate images data

In [8]:
data,data_test = gen_frm_pic.generate_5()

## Split each pixel data to RGB channels

In [12]:
# red channel data
r_data = []
# green channel data
g_data = []
# blue channel data
b_data = []
for image in data:
    print(image[0][0])
    for i in range(3):
        if i == 0:
            r_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(r_temp, INDEX_TRGT, INDEX_END)
            r_data.append(r_temp)

        if i == 1:
            g_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(g_temp, INDEX_TRGT, INDEX_END)
            g_data.append(g_temp)
            
        if i == 2:
            b_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(b_temp, INDEX_TRGT, INDEX_END)
            b_data.append(b_temp)
r_test = []
g_test = []
b_test = []
for image in data_test:
    for i in range(5):
        if i == 0:
            r_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(r_temp, INDEX_TRGT, INDEX_END)
            r_test.append(r_temp)

        if i == 1:
            g_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(g_temp, INDEX_TRGT, INDEX_END)
            g_test.append(g_temp)
            
        if i == 2:
            b_temp = image[i].reshape(ROW_SHAPE)[0]
            util_functions.swap(b_temp, INDEX_TRGT, INDEX_END)
            b_test.append(b_temp)
data = [r_data, g_data, b_data]

[11 13  0]


ValueError: cannot reshape array of size 15 into shape (1,25)

## Split data to train, test and vailation data

In [ ]:
# for red channel
train_r_data = r_data[0:1000]
test_r_data = r_data[700:1000]

# for green channel
train_g_data = g_data[0:700]
test_g_data = r_data[700:1000]

# for blue channel
train_b_data = b_data[0:700]
test_b_data = b_data[700:1000]

In [ ]:
# assuming 3 regressors will give good accuracy
# prepering the data for each regressor
x_train_r = []
x_train_g = []
x_train_b = []

y_train_r = []
y_train_g = []
y_train_b = []

x_test_r = []
x_test_g = []
x_test_b = []

y_test_r = []
y_test_g = []
y_test_b = []

for i in range(1000):
    
    x_test_r.append(r_test[i][0:INDEX_END].tolist())
    x_test_g.append(g_test[i][0:INDEX_END].tolist())
    x_test_b.append(b_test[i][0:INDEX_END].tolist())

    y_test_r.append([r_test[i][INDEX_END]])
    y_test_g.append([g_test[i][INDEX_END]])
    y_test_b.append([b_test[i][INDEX_END]])

for i in range(700):
    x_train_r.append(train_r_data[i][0:INDEX_END].tolist())
    y_train_r.append([train_r_data[i][INDEX_END]])
x_testers = [x_test_r,x_test_g,x_test_b]
y_testers = [y_test_r,y_test_g,y_test_b]

## Model

In [ ]:
features = 24 # each pixel
hidden_layer_nodes = 15
loss_plot = []
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W1 = tf.Variable(tf.truncated_normal([features, hidden_layer_nodes]))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden_layer_nodes]))
z1 = tf.add(tf.matmul(x,W1),b1)
a1 = tf.nn.relu(z1) # using relu function
W2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes,1]))
b2 = tf.Variable(0.)
z2 = tf.matmul(a1,W2) + b2
st_w = []
nd_w = []

loss = tf.reduce_mean(tf.pow(z2 - y_, 2))
update = tf.train.AdamOptimizer(0.001).minimize(loss) # using adam optimizer

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(0,10000):
        sess.run(update, feed_dict = {x:x_train_r, y_:y_train_r})
        st_w.append(sess.run(W1))
        nd_w.append(sess.run(W2))
        if i > 1000:
            loss_plot.append(loss.eval(session=sess, feed_dict = {x:x_train_r, y_:y_train_r}))

In [ ]:
print(np.sqrt(loss.eval(session=sess, feed_dict = {x:x_train_r, y_:y_train_r})))

In [ ]:
plt.pyplot.plot(loss_plot)

first layer weights over time

In [ ]:
d = np.array(np.array(st_w).transpose()[0]).transpose()
print(d[900])
plt.pyplot.plot(d)
plt.pyplot.ylabel('first layer over time')
plt.pyplot.plot(d)

In [ ]:
d = np.array(np.array(nd_w).transpose()[0]).transpose()
print(d[900])
plt.pyplot.plot(d)
plt.pyplot.ylabel('first layer over time')
plt.pyplot.plot(d)

In [ ]:
test_x_r = []
test_y_r = []
for i in range(len(test_r_data)):
    test_x_r.append(test_r_data[i][0:INDEX_END].tolist())
    test_y_r.append([test_r_data[i][INDEX_END].tolist()])
test_x_g = []
test_y_g = []
for i in range(len(test_g_data)):
    test_x_g.append(test_g_data[i][0:INDEX_END].tolist())
    test_y_g.append([test_g_data[i][INDEX_END].tolist()])
test_x_b = []
test_y_b = []
for i in range(len(test_g_data)):
    test_x_b.append(test_b_data[i][0:INDEX_END].tolist())
    test_y_b.append([test_b_data[i][INDEX_END].tolist()])
    
print("sqaured test error on the red channel:", np.sqrt(loss.eval(session=sess, feed_dict = {x:x_testers[0], y_:y_testers[0]})))
print("sqaured test error on the red channel:", np.sqrt(loss.eval(session=sess, feed_dict = {x:x_testers[1], y_:y_testers[1]})))
print("sqaured test error on the red channel:", np.sqrt(loss.eval(session=sess, feed_dict = {x:x_testers[2], y_:y_testers[2]})))